In [ ]:
from google.colab import drive
drive.mount('/content/drive')  

Mounted at /content/drive


# Data Pipeline

In [ ]:
import glob
import cv2
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers

In [ ]:
main_train_paths=[]
main_test_paths=[]
main_val_paths=[]
for i in range(1,11):
  path=f'/content/drive/MyDrive/Object_detection/GroceryDataset_part1/ProductImages/{i}/*.jpg'
  paths=glob.glob(path)
  part1=int(len(paths)*0.9)
  train_paths=paths[:part1]
  test_paths=paths[part1:]
  part2=int(len(train_paths)*0.8)
  val_paths=train_paths[part2:]
  train_paths=train_paths[:part2]
  main_train_paths+=train_paths
  main_val_paths+=val_paths
  main_test_paths+=test_paths
  
train_image_count=len(main_train_paths)
val_image_count=len(main_val_paths)
test_image_count=len(main_test_paths)
print(train_image_count,test_image_count,val_image_count)

2659 376 671


In [ ]:
for i in range(1,11):
  path=f'/content/drive/MyDrive/Object_detection/GroceryDataset_part1/aug_images/{i}/*.jpg'
  paths=glob.glob(path)
  part1=int(len(paths)*0.9)
  train_paths=paths[:part1]
  test_paths=paths[part1:]
  part2=int(len(train_paths)*0.8)
  val_paths=train_paths[part2:]
  train_paths=train_paths[:part2]
  main_train_paths+=train_paths
  main_val_paths+=val_paths
  main_test_paths+=test_paths
  
train_image_count=len(main_train_paths)
val_image_count=len(main_val_paths)
test_image_count=len(main_test_paths)
print(train_image_count,test_image_count,val_image_count)

5179 736 1301


In [ ]:
train_ds = tf.data.Dataset.list_files(main_train_paths, shuffle=False)
train_ds = train_ds.shuffle(train_image_count, reshuffle_each_iteration=False)

In [ ]:
val_ds = tf.data.Dataset.list_files(main_val_paths, shuffle=False)
val_ds = val_ds.shuffle(val_image_count, reshuffle_each_iteration=False)

In [ ]:
test_ds = tf.data.Dataset.list_files(main_test_paths, shuffle=False)
test_ds = test_ds.shuffle(test_image_count, reshuffle_each_iteration=False)

In [ ]:
for i in train_ds.take(5):
  print(i.numpy())

b'/content/drive/MyDrive/Object_detection/GroceryDataset_part1/ProductImages/5/B5_N9.jpg'
b'/content/drive/MyDrive/Object_detection/GroceryDataset_part1/aug_images/2/FlipUpDown-B2_N264.jpg'
b'/content/drive/MyDrive/Object_detection/GroceryDataset_part1/ProductImages/9/B9_N168.jpg'
b'/content/drive/MyDrive/Object_detection/GroceryDataset_part1/aug_images/8/Rotation-B8_N292.jpg'
b'/content/drive/MyDrive/Object_detection/GroceryDataset_part1/aug_images/7/FlipUpDown-B7_N236.jpg'


# Create Augmented Data

In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomRotation(0.2),
])

def flip_ud(img):
  img = tf.image.random_flip_up_down(img)
  return img 
def sat_img(img): 
  img = tf.image.adjust_saturation(img, 3)
  return img 
def cen_img(img):
  img = tf.image.central_crop(img,central_fraction=0.85)
  return img

def aug_process_path_flip_ud(file_path):
  img = tf.io.read_file(file_path)
  img = tf.image.decode_jpeg(img, channels=3) 
  img=flip_ud(img)
  return img,file_path

def aug_process_path_sat(file_path):
  img = tf.io.read_file(file_path)
  img = tf.image.decode_jpeg(img, channels=3) 
  img=sat_img(img)
  return img,file_path

def aug_process_path_rot(file_path):
  img = tf.io.read_file(file_path)
  img = tf.image.decode_jpeg(img, channels=3) 
  img=data_augmentation(img)
  return img,file_path

def aug_process_path_cet(file_path):
  img = tf.io.read_file(file_path)
  img = tf.image.decode_jpeg(img, channels=3) 
  img=cen_img(img)
  return img,file_path

aug_train_ds1 = train_ds.map(aug_process_path_flip_ud, num_parallel_calls=AUTOTUNE)
aug_train_ds2 = train_ds.map(aug_process_path_sat, num_parallel_calls=AUTOTUNE)
aug_train_ds3 = train_ds.map(aug_process_path_cet, num_parallel_calls=AUTOTUNE)
aug_train_ds4 = train_ds.map(aug_process_path_rot, num_parallel_calls=AUTOTUNE)

import matplotlib.pyplot as plt
import os
# Total: 2664
base_dir='/content/drive/MyDrive/Object_detection/GroceryDataset_part1/aug_images'
c=0
for image,fp in aug_train_ds2.as_numpy_iterator():
  c+=1
  if c%100==0:
    print(c)
  img_path=fp.decode('utf-8')
  img_name=img_path.split('/')[-1].split('.')[0]
  class_name=img_path.split('/')[-2]
  if not os.path.exists(f'{base_dir}/{class_name}'):
    os.mkdir(f'{base_dir}/{class_name}')
  else:
    if len(glob.glob(f'{base_dir}/{class_name}/*.jpg'))<=350:
      cv2.imwrite(f'{base_dir}/{class_name}/Saturation-{img_name}.jpg',image)
    else:
      pass

# Create Data Loader

In [ ]:
def get_label(file_path):
  label_name=tf.strings.split(file_path, os.path.sep)[-2] #split('.')[0]
  if label_name=='1':
      label=[0]
  elif label_name=='2':
      label=[1]
  elif label_name=='3':
      label=[2]
  elif label_name=='4':
      label=[3]
  elif label_name=='5':
      label=[4]
  elif label_name=='6':
      label=[5]
  elif label_name=='7':
      label=[6]
  elif label_name=='8':
      label=[7]
  elif label_name=='9':
      label=[8]
  else:
      label=[9]
  return tf.expand_dims(label,axis=1)

def resize_img(img,t_height,t_width):
  img = tf.image.decode_jpeg(img, channels=3) 
  img=img/255
  img=tf.image.resize_with_pad(img,t_height, t_width)
  return img

def process_path(file_path):
  label = get_label(file_path)
  img = tf.io.read_file(file_path)
  img = resize_img(img,t_height,t_width)
  img=tf.expand_dims(img, axis=0)
  return img, label

def test_process_path(file_path):
  label = get_label(file_path)
  img = tf.io.read_file(file_path)
  img = resize_img(img,t_height,t_width)
  img=tf.expand_dims(img, axis=0)
  return img, label

In [ ]:
t_height,t_width=352,208
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
new_train_ds = train_ds.map(process_path, num_parallel_calls=AUTOTUNE)
new_val_ds = val_ds.map(process_path, num_parallel_calls=AUTOTUNE)
new_test_ds = test_ds.map(test_process_path, num_parallel_calls=AUTOTUNE)

In [ ]:
import matplotlib.pyplot as plt

for image, label in new_train_ds.take(5):
  print("Image shape: ", image.numpy().shape)
  fig=plt.figure()
  plt.imshow(image.numpy()[0])
  print("Label: ", label.numpy())

In [ ]:
for path in train_paths:
  if path.split('/')[-1].split('.')[-1]!='jpg':
    print(path)

# Model Training

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop

pre_trained_model = InceptionV3(input_shape = (t_height,t_width, 3), 
                                include_top = False, 
                                weights = 'imagenet')


for layer in pre_trained_model.layers:
     layer.trainable = False
        
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

x = layers.Flatten()(last_output)
# x = layers.Dense(1024, activation='relu')(x)
# x = layers.Dropout(0.2)(x)                  
x = layers.Dense(10, activation='softmax')(x)           

model = Model(pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(learning_rate=0.001), 
              loss = 'sparse_categorical_crossentropy', 
              metrics = ['acc'])

last layer output shape:  (None, 20, 11, 768)


In [ ]:
checkpoint_filepath = '/content/drive/MyDrive/Object_detection/Item_classifier_model'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)

EPOCHS=1
history=model.fit(new_train_ds,batch_size=32,validation_data=new_val_ds,epochs=EPOCHS, callbacks=[model_checkpoint_callback])

5179/5179 [==============================] - 572s 94ms/step - loss: 0.5026 - acc: 0.9344 - val_loss: 0.0138 - val_acc: 0.9977


In [ ]:
history=model.fit(new_train_ds,batch_size=32,validation_data=new_val_ds,epochs=EPOCHS, callbacks=[model_checkpoint_callback])

5179/5179 [==============================] - 169s 33ms/step - loss: 0.0023 - acc: 0.9992 - val_loss: 0.0112 - val_acc: 0.9977


In [ ]:
model.save("/content/drive/MyDrive/Object_detection/Aug_item_classifier_model.h5")

In [ ]:
history=model.fit(new_train_ds,batch_size=32,validation_data=new_val_ds,epochs=EPOCHS, callbacks=[model_checkpoint_callback])

5179/5179 [==============================] - 172s 33ms/step - loss: 1.8687e-04 - acc: 1.0000 - val_loss: 0.0014 - val_acc: 0.9992


In [ ]:
model.save("/content/drive/MyDrive/Object_detection/Aug_item_classifier_model1.h5")

In [ ]:
history=model.fit(new_train_ds,batch_size=32,validation_data=new_val_ds,epochs=EPOCHS, callbacks=[model_checkpoint_callback])

5179/5179 [==============================] - 169s 33ms/step - loss: 1.5868e-07 - acc: 1.0000 - val_loss: 2.2260e-04 - val_acc: 1.0000


In [ ]:
model.save("/content/drive/MyDrive/Object_detection/Aug_item_classifier_model2.h5") 

# Evaluation

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

y_actual = []
for image, label in new_test_ds:
  y_actual.append(label.numpy().reshape(-1))
y_actual = np.array(y_actual)

In [ ]:
from keras import models
model=models.load_model('/content/drive/MyDrive/Object_detection/Item_classifier_model')

In [ ]:
model.evaluate(new_test_ds)

In [ ]:
model=models.load_model('/content/drive/MyDrive/Object_detection/item_classifier_model.h5')

In [ ]:
model.evaluate(new_test_ds)

In [ ]:
pred=model.predict(new_test_ds)